<a href="https://colab.research.google.com/github/Kabongosalomon/RDC-Mobongoli/blob/main/jw300_utils/building_global_test_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Setting Up the data

Downloading the global test set is simple,
we need to set english and your target as source and target language, then we find the intersection of the english test set with the target corpus after that we get the corresponding target sentences from the target corpus.

In [2]:
%%capture
!pip install opustools-pkg

# SET THE LANGUAGE CODE and other variables.

You need to change the value below for your language!

The language codes from the [JW300 corpus website](https://object.pouta.csc.fi/OPUS-JW300/v1/languages.json) are: 
```
{
    "language": "French - Français",
    "language_en": "French",
    "language_native": "Français",
    "language_short": "fr",
    "url": "https://wol.jw.org/fr/wol/pref/r30/lp-f?newrsconf=r30&newlib=lp-f&url="
}, 

{
    "language": "Lingala - Lingala",
    "language_en": "Lingala",
    "language_native": "Lingala",
    "language_short": "ln",
    "url": "https://wol.jw.org/ln/wol/pref/r126/lp-li?newrsconf=r126&newlib=lp-li&url="
},

{
    "language": "Tshiluba - Tshiluba",
    "language_en": "Tshiluba",
    "language_native": "Tshiluba",
    "language_short": "lua",
    "url": "https://wol.jw.org/lua/wol/pref/r477/lp-sh?newrsconf=r477&newlib=lp-sh&url="
},

{
    "language": "Kikongo - Kikongo",
    "language_en": "Kikongo",
    "language_native": "Kikongo",
    "language_short": "kwy",
    "url": "https://wol.jw.org/kwy/wol/pref/r128/lp-kg?newrsconf=r128&newlib=lp-kg&url="
},

{
    "language": "Swahili (Congo) - Kiswahili (Congo)",
    "language_en": "Swahili (Congo)",
    "language_native": "Kiswahili (Congo)",
    "language_short": "swc",
    "url": "https://wol.jw.org/swc/wol/pref/r143/lp-zs?newrsconf=r143&newlib=lp-zs&url="
  },
```
Already-created test sets: https://raw.githubusercontent.com/ai-drc/RDC-Mobongoli/main/jw300_utils/test/

In [31]:
import os
source_language = "fr"
target_language = "ln" # TODO: CHANGE THIS TO YOUR LANGUAGE! "ha" is hausa. See the language codes at https://opus.nlpl.eu/JW300.php
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# No need to use gdrive since we are training on gcp
!mkdir -p "$src-$tgt-$tag"
os.environ["gdrive_path"] = "%s-%s-%s" % (source_language, target_language, tag) # saving directly on the vm

In [32]:
!echo $gdrive_path

fr-ln-baseline


#### Downloading the corpus data

for precaution , am removing the old data

In [33]:
!rm -f w300.$src jw300.$tgt JW300_latest_xml_$src-$tgt.xml.gz JW300_latest_xml_$src-$tgt.xml JW300_latest_xml_$src.zip  JW300_latest_xml_$tgt.zip test.fr-any.fr

In [34]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/fr-ln.xml.gz not found. The following files are available for downloading:

   6 MB https://object.pouta.csc.fi/OPUS-JW300/v1b/xml/fr-ln.xml.gz
 278 MB https://object.pouta.csc.fi/OPUS-JW300/v1b/xml/fr.zip
  60 MB https://object.pouta.csc.fi/OPUS-JW300/v1b/xml/ln.zip

 345 MB Total size
./JW300_latest_xml_fr-ln.xml.gz ... 100% of 6 MB
./JW300_latest_xml_fr.zip ... 100% of 278 MB
./JW300_latest_xml_ln.zip ... 100% of 60 MB


In [35]:
! wget https://raw.githubusercontent.com/ai-drc/RDC-Mobongoli/main/jw300_utils/test/test.fr-any.fr

  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

--2021-07-20 14:49:55--  https://raw.githubusercontent.com/ai-drc/RDC-Mobongoli/main/jw300_utils/test/test.fr-any.fr
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 377235 (368K) [text/plain]
Saving to: ‘test.fr-any.fr’

test.fr-any.fr      100%[===================>] 368.39K  --.-KB/s    in 0.06s   

2021-07-20 14:49:56 (6.00 MB/s) - ‘test.fr-any.fr’ saved [377235/377235]



In [36]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.fr-any.fr"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3974 global test sentences to filter from the training/dev data.


In [37]:
!ls

drive			    JW300_latest_xml_fr-lua.xml  jw300.lua
fr-ln-baseline		    JW300_latest_xml_fr.zip	 sample_data
fr-lua-baseline		    JW300_latest_xml_ln.zip	 test.fr-any.fr
jw300.fr		    JW300_latest_xml_lua.zip	 test.fr-lua.fr
JW300_latest_xml_fr-ln.xml  jw300.ln			 test.lua-any.lua


#### Building the corpus

In the 2 cells below you can check if the 2 datasets are aligned. Even if you don't speak the language you can get a sense, especially with similar words, punctuation, and so forth.

In [38]:
! head -5 jw300.fr

Qui veut être millionnaire ?
IL SEMBLE que ce soit là le désir de tout un chacun , ou presque .
Or la solution la plus simple , dans l’esprit du public , est de gagner à la loterie ou au loto sportif * .
Flattant les désirs du grand nombre — et convoitant les excédents qui reviendront à l’État — , de Moscou à Madrid , de Manille à Mexico , les gouvernements parrainent des loteries d’État qui peuvent faire gagner l’équivalent de plusieurs centaines de millions de francs français .
Quelques joueurs deviennent effectivement millionnaires .


In [39]:
! head -5 jw300.$tgt

Nani alingi kozala milionere ?
EYANO emonani lokola ete , wana ezali mposa ya moto na moto to pene na bato nyonso .
Nzokande , na makanisi ya bato , nzela ya pɛtɛɛ mpo na kozwa yango ezali kolónga na loterie to na momekano ya kosakola liboso équipe ya ndembo oyo ekolónga .
Kolamusáká mposa ya bato mingi ​ — mpe koluláká kozwa misolo oyo Leta akozwa likoló ​ — kolongwa Moscou kino Madrid , kolongwa Manille kino Mexico , baguvernema bazali kopesa lisungi na loterie esalemi na Leta , kati na yango balóngi bakoki kozwa nkámá mingi ya bamilió ya badolare .
Mwa babɛti na yango bazali mpenza kokóma bamilionere .


In [40]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language  ## source language is english
target_file = 'jw300.' + target_language ## Target is whatever you set. For our example it was ha, so jw300.ha
target_test = {}
source = []
target = []
english_sentences_in_global_test_set = {}  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as src_f:
    for i, line in enumerate(src_f):
        # Skip sentences that are contained in the test set and add them into the new frencg test
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            # Here is the intersection with the global test set
            english_sentences_in_global_test_set[i] = line.strip()           
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in english_sentences_in_global_test_set.keys():
            target.append(line.strip())
        else:
            #Collecting the aligned test sentences
            target_test[j] = line.strip()
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(english_sentences_in_global_test_set.keys()), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(10)

Loaded data and skipped 6707/590525 lines since contained in test set.


,source_sentence,target_sentence
0,Qui veut être millionnaire ?,Nani alingi kozala milionere ?
1,IL SEMBLE que ce soit là le désir de tout un c...,"EYANO emonani lokola ete , wana ezali mposa ya..."
2,"Or la solution la plus simple , dans l’esprit ...","Nzokande , na makanisi ya bato , nzela ya pɛtɛ..."
3,Flattant les désirs du grand nombre — et convo...,Kolamusáká mposa ya bato mingi ​ — mpe kolulák...
4,Quelques joueurs deviennent effectivement mill...,Mwa babɛti na yango bazali mpenza kokóma bamil...
5,"Un Anglais , qui avait parié pendant 25 ans su...",Mongelesi moko oyo azalaki kosala momekano na ...
6,Une mise équivalant à moins de 3 francs frança...,Abɛtaki bobele na mosolo mokokani na franka 3 ...
7,Plus spectaculaire encore est le cas de cette ...,Oyo ekamwisi mpenza ezali likambo ya mwasi oyo...
8,Mais ce ne sont là que des exceptions .,Kasi baoyo bazali kolónga boye bazali sé moke ...
9,Plus représentatif est cet employé de bureau e...,Ndakisa emonisi yango malamu ezali mosali na b...


## Check a random item!
Let's pick one of the keys in the dictionary at random and check it. 

In [41]:
import random
keys_in_target_test = list(target_test.keys())
print(type(keys_in_target_test))
random_key = random.choice(keys_in_target_test)
print(f"The random key we picked was {random_key}")

<class 'list'>
The random key we picked was 382297


In [42]:
target_test[random_key]

'1 , 2 .'

In [43]:
english_sentences_in_global_test_set[random_key]

'1 , 2 .'

Do the two look like they line up? 

## Check several rows at the tail end

Let's get a sample from the end of the dataset

In [44]:
target_test_set = pd.DataFrame(zip(target_test.values(), english_sentences_in_global_test_set.values()), columns=['target_equivalent', 'english_equivalent'])

In [45]:
target_test_set = target_test_set.reset_index()

In [46]:
target_test_set = target_test_set.set_index("index")

In [47]:
target_test_set.tail()

,target_equivalent,english_equivalent
index,,
6702,"Sikoyo , nani aleki na mayele : moto oyo asalá...","Alors , qui est le plus intelligent : le créat..."
6703,"13 , 14 .","13 , 14 ."
6704,"Yango wana , ezali na ntina mingi ete baboti b...",C’est pour cela que c’est important que les pa...
6705,Tiká bana na yo bámona ete Yehova azali mpenza...,Montre - ​ leur que Jéhovah est vraiment réel ...
6706,Yango ekómisaki makasi kondima na ye epai ya N...,C’est excellent pour sa foi en Dieu et en la B...


Removing duplicates from english and target set

In [48]:
target_test_set = target_test_set.drop_duplicates(subset='target_equivalent')

In [49]:
target_test_set = target_test_set.drop_duplicates(subset='english_equivalent')

In [50]:
target_test_set.head()

,target_equivalent,english_equivalent
index,,
0,Mpo na nini ?,Pourquoi ?
1,Oyo ezali mobeko moleki monene mpe oyo na libo...,C’est là le plus grand et le premier commandem...
6,Lamuká !,Réveillez - vous !
11,Sapolsky mpe E .,Non .
18,Oyo ezali mobeko moleki monene mpe ya liboso .,’ C’est là le plus grand et le premier command...


In [51]:
target_test_set.shape

(2933, 2)

In [52]:
target_test_set.loc[~target_test_set.english_equivalent.isin(en_test_sents)]

,target_equivalent,english_equivalent
index,,


## Write out target-language test set file
In our example, we should have `test.ln-any.ln`, but it will be different for you if you picked a different code.

In [53]:
target_test_filename = f"test.{target_language}-any.{target_language}"
print(target_test_filename)

test.ln-any.ln


## Write out English-language test set file
In our example, we should have `test.fr-ln.fr`, but it will be different for you if you picked a different code.

**Make sure the data lines up in the two files!**
The first line of each file should be translations of each other.


In [54]:

with open(target_test_filename, "w") as test_tgt_any_tgt:
    test_tgt_any_tgt.write("\n".join(target_test_set.target_equivalent))

In [55]:
!head -5 test.$tgt-any.$tgt

Mpo na nini ?
Oyo ezali mobeko moleki monene mpe oyo na liboso .
Lamuká !
Sapolsky mpe E .
Oyo ezali mobeko moleki monene mpe ya liboso .


In [56]:
source_test_filename = f"test.fr-{target_language}.fr"
print(f"saving english aligned sentences to {source_test_filename}")
with open(source_test_filename, "w") as test_en_tgt_en:
    test_en_tgt_en.write("\n".join(target_test_set.english_equivalent))
!ls -al

saving english aligned sentences to test.fr-ln.fr
total 576272
drwxr-xr-x 1 root root      4096 Jul 20 14:49 .
drwxr-xr-x 1 root root      4096 Jul 20 14:30 ..
drwxr-xr-x 4 root root      4096 Jul 16 13:19 .config
drwx------ 6 root root      4096 Jul 20 14:31 drive
drwxr-xr-x 2 root root      4096 Jul 20 14:46 fr-ln-baseline
drwxr-xr-x 2 root root      4096 Jul 20 14:33 fr-lua-baseline
-rw-r--r-- 1 root root  62642691 Jul 20 14:49 jw300.fr
-rw-r--r-- 1 root root  34104082 Jul 20 14:46 JW300_latest_xml_fr-ln.xml
-rw-r--r-- 1 root root  18757876 Jul 20 14:34 JW300_latest_xml_fr-lua.xml
-rw-r--r-- 1 root root 285126296 Jul 20 14:47 JW300_latest_xml_fr.zip
-rw-r--r-- 1 root root  61546014 Jul 20 14:47 JW300_latest_xml_ln.zip
-rw-r--r-- 1 root root  32608568 Jul 20 14:34 JW300_latest_xml_lua.zip
-rw-r--r-- 1 root root  61075229 Jul 20 14:49 jw300.ln
-rw-r--r-- 1 root root  32756386 Jul 20 14:36 jw300.lua
drwxr-xr-x 1 root root      4096 Jul 16 13:20 sample_data
-rw-r--r-- 1 root root    377

In [57]:
!head -5 test.fr-$tgt.fr

Pourquoi ?
C’est là le plus grand et le premier commandement .
Réveillez - vous !
Non .
’ C’est là le plus grand et le premier commandement .


## One last check to see if the two files are aligned

Let's just get one more sample! Let's take from the end this time

In [58]:
!echo "test.fr-$tgt.fr"
!tail -5 test.fr-$tgt.fr
!echo
!echo "**********************"
!echo "test.$tgt-any.$tgt"
!echo "**********************"
!tail -5 test.$tgt-any.$tgt

test.fr-ln.fr
Et qu’est - ​ ce qui est plus joli : le bruit d’un avion ou le chant d’un oiseau ?
Alors , qui est le plus intelligent : le créateur des avions ou le Créateur des oiseaux ?
C’est pour cela que c’est important que les parents n’arrêtent jamais d’enseigner .
Montre - ​ leur que Jéhovah est vraiment réel pour toi .
C’est excellent pour sa foi en Dieu et en la Bible . »
**********************
test.ln-any.ln
**********************
Mpe okoloba nini mpo na makɛlɛlɛ ya mpɛpɔ ná koyemba ya ndɛkɛ ?
Sikoyo , nani aleki na mayele : moto oyo asalá mpɛpɔ to Mozalisi ya bandɛkɛ ? ”
Yango wana , ezali na ntina mingi ete baboti bákoba koteya bana na bango . ”
Tiká bana na yo bámona ete Yehova azali mpenza solo na miso na yo .
Yango ekómisaki makasi kondima na ye epai ya Nzambe mpe na Biblia . ”